In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np

torch.__version__


# Data Preparing and loading

In [ ]:
# Create known parameters
weight = 0.7
bias = 0.3

# in linear reqgression, weight is 'b' and bias is 'a' from
# Y = a + bX

start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias
#unsqueeze adds one more bracket

In [ ]:
# Splitting in training(60-80) and test(20-40) set

train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test,y_test = X[train_split:], y[train_split:]


In [ ]:
def plot_predictions(train_data=X_train, train_lables=y_train, test_data=X_test, test_labels=y_test, predictions=None):
    #plots training daat, test data and compare predictiosn
    plt.figure(figsize=(10,7))
    #Plot training data in blue
    plt.scatter(train_data, train_lables, c="b", s=4, label="Training data")
    #Plot test data in green
    plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")
    # plot predictions if exist

    if predictions is not None:
        plt.scatter(test_data, predictions, c="r", s=4, label="Prediction")
    #legend
        
    plt.legend(prop={"size": 14});


In [ ]:
plot_predictions()

In [ ]:
# create a model with param as A and B
# Create linear regression model class

class LinearRegressionModel(nn.Module): #inherits from nn.module
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1, requires_grad=True,dtype=torch.float))


    # Forward Method to define computation in Model
    # x is input data
    def forward(self, x: torch.Tensor) -> torch.Tensor: 
        return self.weights * x + self.bias




What model does:
*  start with random values of weight and bias
* Look at training data and adjust the random vaues to get closer to ideal values


How it does?
1. Gradient Descent
2. Back propogation



### TOrch model building essentials

* torch.nn - contains all building blocks for Neural Network

* toech.nn.Parameter - Stores tensors that can be used with nn.Module. If requires_grad=True gradients (used for updating model parameters via gradient descent) are calculated automatically, this is often referred to as "autograd".

* torch.nn.Module - The base class for all neural network modules, all the building blocks for neural networks are subclasses. If you're building a neural network in PyTorch, your models should subclass nn.Module. Requires a f`orward()` method be implemented.

* torch.optim - Contains various optimization algorithms (these tell the model parameters stored in nn.Parameter how to best change to improve gradient descent and in turn reduce the loss).

* `def forward()` All nn.Module subclasses require you to override forward(), this is what happens in forward computation


In [ ]:
torch.manual_seed(57)

model_0 = LinearRegressionModel()

list(model_0.parameters())


In [ ]:
model_0.state_dict()

In [ ]:
# Make predictions using torch.inference_mode()
# When we pass data to our model, it'll go through the model's forward() method and produce a result using the computation we've defined.

with torch.inference_mode():
    y_preds = model_0(X_test)

y_preds

#inference reduces gradient

In [ ]:
plot_predictions(predictions=y_preds)

In [ ]:
# idea of training is to move from randomm param to some known params

# One way to measure how poor or wrong your model predictions are is to use a loss func

# Loss func is also called cost func or criterion

# 5:45:41
# loss func is measure of how wrong your predictions are compared to OG

# optimizer: Takes into account loss of model and adjust the params

# For torch, we need a training loop and a testing loop

In [ ]:
# set up Loss function (pick fron many)

loss_fn = nn.L1Loss()


# setup an optimizer (pick fron many)
# stochastic gradient descent

# LR := learning rate , most imp param
# smol Lr, smol change in param

optimizer = torch.optim.SGD(params=model_0.parameters(), 
                            lr=0.01) 

#### Training Loop (and testing) in Torch

0. Loop through data
1. Forward pass (involve data moving through model's `forward()` func) := called as forward propogation
2. Calculate the loss (compare forward pass prediction to ground truth lable)
3. OPtimizer zero grad
4. Loss backward - move backwards through network to calculate gradients of each param of model wrt loss (**back propogation**)

5. Optimizer step := use to adjust model's param to try and improve loss  (**gradient descent**)

In [ ]:
torch.manual_seed(57)

# epoch is one loop through data, hyperparameter
epochs = 250

epoch_count = []
loss_values = []
test_loss_values = []

# Training

# 0.
for epoch in range(epochs):
    #train mode sets all param that require gradient to gradient
    model_0.train()

    # 1.
    y_pred = model_0(X_train)

    # 2.
    loss = loss_fn(y_pred, y_train)
    # 3. 
    optimizer.zero_grad() # make it zero, because it accumulates garbage value


    # 4.
    loss.backward()

    # 5.

    optimizer.step() 

    # tessting
    model_0.eval() # turns off gradient tracking, not needed for testing

    with torch.inference_mode():
        #1. do forward pass
        test_pred = model_0(X_test)

        #2. calculate test loss
        test_loss = loss_fn(test_pred, y_test)

    

    if epoch % 10 == 0:
        epoch_count.append(epoch)
        loss_values.append(loss)
        test_loss_values.append(test_loss)
        
        print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")
        # print state
        print(model_0.state_dict())



In [ ]:
model_0.state_dict()

np.array(torch.tensor(loss_values).numpy())


In [ ]:
epoch_count, loss_values, test_loss_values

plt.plot(epoch_count, np.array(torch.tensor(loss_values).numpy()), label="Traain Loss")
plt.plot(epoch_count, test_loss_values, label="TEst Loss")
plt.title("training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()

In [ ]:
with torch.inference_mode():
    y_preds_new = model_0(X_test)

plot_predictions(predictions=y_preds_new)

### Save a model

1. `torch.save()` := saves in python .pkl format
2. `torch.load()` := load a saved torch model
3. `torch.nn.Module.load_state_dict()` := loadds model's saved dict


In [ ]:
from pathlib import Path

# create model path
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# create save path
MODEL_NAME = "01_torch_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME


# save model

print(MODEL_SAVE_PATH)
torch.save(obj=model_0.state_dict(), f=MODEL_SAVE_PATH)


In [ ]:
# Load model

loaded_model_0 = LinearRegressionModel()
print(loaded_model_0.state_dict())

loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
print(loaded_model_0.state_dict())

In [ ]:
# make prediction

loaded_model_0.eval()
with torch.inference_mode():
    loaded_model_preds = loaded_model_0(X_test)

loaded_model_preds


# Revisit

In [ ]:
class LinearRegressionModelV2(nn.Module):
    def __init__(self) :
        super().__init__()

        # use this for creating model param
        # also called linear transform, fully connected layer, dense layer
        self.linear_layer = nn.Linear(in_features=1,
                                      out_features=1)
        
    def forward(self, x: torch.Tensor ) -> torch.Tensor:
        return self.linear_layer(x)
    

torch.manual_seed(57)
model_1 = LinearRegressionModelV2()
model_1, model_1.state_dict()

In [ ]:
# training

# loss func
loss_fn = nn.L1Loss() #same as MAE

# optimizer
optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.01)

In [ ]:
# Training Loop
torch.manual_seed(57)

epochs = 500

for epoch in range(epochs):
    model_1.train()

    # Forward pass
    y_pred = model_1(X_train)


    # calculate loss
    loss = loss_fn(y_pred, y_train)

    # optimize zero grad, aka remove waste
    optimizer.zero_grad()

    # Back propogation
    loss.backward()

    # optimizer step
    optimizer.step()


    # Testing
    model_1.eval()
    with torch.inference_mode():
        test_pred = model_1(X_test)

        test_loss = loss_fn(test_pred, y_test)

    

    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")
        print(model_1.state_dict())

In [ ]:


# Turn model into evaluation mode
model_1.eval()

# Make predictions on the test data
with torch.inference_mode():
    y_preds = model_1(X_test)
y_preds


In [ ]:
plot_predictions(predictions=y_preds)

In [ ]:
# saving model

from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
MODEL_NAME = "01_pytorch_workflow_model_1.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_1.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

In [ ]:


# Instantiate a fresh instance of LinearRegressionModelV2
loaded_model_1 = LinearRegressionModelV2()

# Load model state dict 
loaded_model_1.load_state_dict(torch.load(MODEL_SAVE_PATH))

In [ ]:


# Evaluate loaded model
loaded_model_1.eval()
with torch.inference_mode():
    loaded_model_1_preds = loaded_model_1(X_test)
y_preds == loaded_model_1_preds
